In [3]:
import org.apache.spark.sql.SparkSession
val spark = SparkSession.builder().getOrCreate()

import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@2e452be


In [4]:
val df = spark.read.option("header", "true").option("inferSchema", "true").csv("../datasets/Netflix_2011_2016.csv")
df.show()

+----------+-----------------+------------------+----------+-----------------+---------+------------------+
|      Date|             Open|              High|       Low|            Close|   Volume|         Adj Close|
+----------+-----------------+------------------+----------+-----------------+---------+------------------+
|2011-10-24|       119.100002|120.28000300000001|115.100004|       118.839996|120460200|         16.977142|
|2011-10-25|        74.899999|         79.390001| 74.249997|        77.370002|315541800|11.052857000000001|
|2011-10-26|            78.73|         81.420001| 75.399997|        79.400002|148733900|         11.342857|
|2011-10-27|        82.179998| 82.71999699999999| 79.249998|80.86000200000001| 71190000|11.551428999999999|
|2011-10-28|        80.280002|         84.660002| 79.599999|84.14000300000001| 57769600|             12.02|
|2011-10-31|83.63999799999999|         84.090002| 81.450002|        82.080003| 39653600|         11.725715|
|2011-11-01|        80.10999

df: org.apache.spark.sql.DataFrame = [Date: string, Open: double ... 5 more fields]


In [5]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [6]:
df.describe().show()

+-------+----------+------------------+------------------+------------------+------------------+--------------------+------------------+
|summary|      Date|              Open|              High|               Low|             Close|              Volume|         Adj Close|
+-------+----------+------------------+------------------+------------------+------------------+--------------------+------------------+
|  count|      1259|              1259|              1259|              1259|              1259|                1259|              1259|
|   mean|      null|230.39351086656092|233.97320872915006|226.80127876251044|  230.522453845909|2.5634836060365368E7|55.610540036536875|
| stddev|      null|164.37456353264244| 165.9705082667129| 162.6506358235739|164.40918905512854| 2.306312683388607E7|35.186669331525486|
|    min|2011-10-24|         53.990001|         55.480001|             52.81|              53.8|             3531300|          7.685714|
|    max|2016-10-24|        708.900017|  

In [7]:
df.columns

res4: Array[String] = Array(Date, Open, High, Low, Close, Volume, Adj Close)


In [8]:
df.head(5)

res5: Array[org.apache.spark.sql.Row] = Array([2011-10-24,119.100002,120.28000300000001,115.100004,118.839996,120460200,16.977142], [2011-10-25,74.899999,79.390001,74.249997,77.370002,315541800,11.052857000000001], [2011-10-26,78.73,81.420001,75.399997,79.400002,148733900,11.342857], [2011-10-27,82.179998,82.71999699999999,79.249998,80.86000200000001,71190000,11.551428999999999], [2011-10-28,80.280002,84.660002,79.599999,84.14000300000001,57769600,12.02])


In [13]:
val df2 = df.withColumn("HV Ratio", df("High") / df("Volume"))

df2: org.apache.spark.sql.DataFrame = [Date: string, Open: double ... 6 more fields]


In [15]:
df.orderBy("High").show(1)

+----------+------------------+---------+-----+---------+--------+---------+
|      Date|              Open|     High|  Low|    Close|  Volume|Adj Close|
+----------+------------------+---------+-----+---------+--------+---------+
|2012-08-03|54.860001000000004|55.480001|52.81|53.909999|37295300| 7.701428|
+----------+------------------+---------+-----+---------+--------+---------+
only showing top 1 row



In [18]:
df.select(mean("Close")).show()

+----------------+
|      avg(Close)|
+----------------+
|230.522453845909|
+----------------+



In [19]:
df.select(max("Volume"), min("Volume")).show()

+-----------+-----------+
|max(Volume)|min(Volume)|
+-----------+-----------+
|  315541800|    3531300|
+-----------+-----------+



In [20]:
import spark.implicits._

import spark.implicits._


In [21]:
df.filter($"Close" < 600).count()

res15: Long = 1218


In [26]:
df.filter($"High" > 500).count()*1.0 / df.count()

res19: Double = 0.04924543288324067


In [27]:
df.select(corr("High", "Volume")).show()

+--------------------+
|  corr(High, Volume)|
+--------------------+
|-0.20960233287942157|
+--------------------+



In [29]:
val df2 = df.withColumn("Year", year(df("Date")))
val df3 = df2. groupBy("Year").max()

df3.select($"Year", $"max(High)").show()

+----+------------------+
|Year|         max(High)|
+----+------------------+
|2015|        716.159996|
|2013|        389.159988|
|2014|        489.290024|
|2012|        133.429996|
|2016|129.28999299999998|
|2011|120.28000300000001|
+----+------------------+



df2: org.apache.spark.sql.DataFrame = [Date: string, Open: double ... 6 more fields]
df3: org.apache.spark.sql.DataFrame = [Year: int, max(Open): double ... 6 more fields]


In [34]:
val df2 = df.withColumn("Month", month(df("Date")))
val df3 = df2. groupBy("Month").mean()

df3.select($"Month", $"avg(Close)").orderBy("Month").show()

+-----+------------------+
|Month|        avg(Close)|
+-----+------------------+
|    1|212.22613874257422|
|    2| 254.1954634020619|
|    3| 249.5825228971963|
|    4|246.97514271428562|
|    5|264.37037614150944|
|    6| 295.1597153490566|
|    7|243.64747528037387|
|    8|195.25599892727263|
|    9|206.09598121568627|
|   10|205.93297300900903|
|   11| 194.3172275445545|
|   12| 199.3700942358491|
+-----+------------------+



df2: org.apache.spark.sql.DataFrame = [Date: string, Open: double ... 6 more fields]
df3: org.apache.spark.sql.DataFrame = [Month: int, avg(Open): double ... 6 more fields]
